In [1]:
from enum import Enum
from datetime import datetime, timedelta
import pandas as pd 
import os 
import re 

class Granularity(Enum):
    """ The possible Granularity to build the OHLC old_data from lob """
    Sec1 = "1S"
    Sec5 = "5S"
    Sec15 = "15S"
    Sec30 = "30S"
    Min1 = "1Min"
    Min5 = "5Min"
    Min15 = "15Min"
    Min30 = "30Min"
    Hour1 = "1H"
    Hour2 = "2H"
    Hour6 = "6H"
    Hour12 = "12H"
    Day1 = "1D"
    Day2 = "2D"
    Day5 = "7D"
    Month1 = "30D"

class OrderEvent(Enum):
    """event types of orderbook"""
    submission = 1
    cancellation = 2
    deletion = 3
    execution_visible = 4
    execution_hidden = 5
    cross_trade = 6
    halt = 7

In [2]:
def orderbook_columns(level: int):
    """ return the column names for the LOBSTER orderbook, acording the input level """
    orderbook_columns = []
    for i in range(1, level + 1):
        orderbook_columns += ["psell" + str(i), "vsell" + str(i), "pbuy" + str(i), "vbuy" + str(i)]
    return orderbook_columns

def message_columns():
    """ return the message columns for the LOBSTER orderbook """
    return ["time", "event_type", "order_id", "size", "price", "direction", "unk"]

In [3]:
def lobster_to_sec_df(message_df, orderbook_df,
                      datetime_start: datetime,
                      granularity: Granularity = Granularity.Sec1,
                      level: int = 10, 
                      add_messages=True):
    """ create a dataframe with midprices, sell and buy for each second

        message_df : a csv df with the messages (lobster old_data format) without initial start lob
        ordebook_df : a csv df with the orderbook (lobster old_data format) without initial start lob
        datetime_start : should be a start date in the message file and orderbook file
        granularity : the granularity to use in the mid-prices computation
        plot : whether print or not the mid_prices
        level : the level of the old_data
        add_messages : if True keep messages along the orderbook data. It does not work with granularity != None
    """
    start_date = datetime_start

    # to be sure that columns are okay
    orderbook_df.columns = orderbook_columns(level)
    message_df.columns = message_columns()

    # convert the time to seconds and structure the df to the input granularity
    orderbook_df["seconds"] = message_df["time"]

    if add_messages and granularity is not None:
        orderbook_df[message_df.columns] = message_df[message_df.columns]
        accepted_orders = [o.value for o in (OrderEvent.execution_visible, OrderEvent.submission, OrderEvent.execution_hidden)]
        orderbook_df = orderbook_df[orderbook_df["event_type"].isin(accepted_orders)]

    orderbook_df["date"] = [start_date + timedelta(seconds=i) for i in orderbook_df["seconds"]]

    if granularity is not None:
        orderbook_df.set_index("date", inplace=True)
        orderbook_df = orderbook_df.resample(granularity.value).first()
        orderbook_df.reset_index(inplace=True)

    orderbook_df = orderbook_df.sort_values(by="date").reset_index(drop=True).copy()
    orderbook_df.drop(columns=['seconds'], inplace=True)

    return orderbook_df.set_index('date')

In [4]:
def read_sub_routine(file_7z: str, first_date: str = "1990-01-01",
                     last_date: str = "2100-01-01",
                     type_file: str = "orderbook",
                     level: int = 10,
                     path: str = "") -> dict:
    """
        :param file_7z: the input file where the csv with old_data are stored
        :param first_date: the first day to load from the input file
        :param last_date: the last day to load from the input file
        :param type_file: the kind of old_data to read. type_file in ("orderbook", "message")
        :param level: the LOBSTER level of the orderbook
        :param path: data path
        :return: a dictionary with {day : dataframe}
    """
    assert type_file in ("orderbook", "message"), "The input type_file: {} is not valid".format(type_file)

    columns = message_columns() if type_file == "message" else orderbook_columns(level)
    # if both none then we automatically detect the dates from the files
    first_date = datetime.strptime(first_date, "%Y-%m-%d")
    last_date = datetime.strptime(last_date, "%Y-%m-%d")

    all_period = {}  # day :  df

    path = path + file_7z
    for file in sorted(os.listdir(path)):
        # read only the selected type of file
        if type_file not in str(file):
            continue

        # read only the old_data between first_ and last_ input dates
        m = re.search(r".*([0-9]{4}-[0-9]{2}-[0-9]{2}).*", str(file))
        if m:
            entry_date = datetime.strptime(m.group(1), "%Y-%m-%d")
            if entry_date < first_date or entry_date > last_date:
                continue
        else:
            print("error for file: {}".format(file))
            continue

        curr = path + '/' + file
        df = pd.read_csv(curr, names=columns)
        # put types
        all_period[entry_date] = df

    return all_period

In [5]:
!ls /home/ema/dev/shocks/data/lobster/_data_dwn_48_332__AAPL_2021-11-01_2022-04-30_10.7z

/home/ema/dev/shocks/data/lobster/_data_dwn_48_332__AAPL_2021-11-01_2022-04-30_10.7z


In [5]:
def from_folder_to_unique_df(file_7z: str, 
                             first_date: str = "1990-01-01",
                             last_date: str = "2100-01-01",
                             plot: bool = False, 
                             level: int = 10,
                             path: str = "",
                             granularity: Granularity = Granularity.Sec1,
                             add_messages = True):
    """ return a unique df with also the label

        add_messages : if True keep messages along the orderbook data. It does not work with granularity != None

    """
    message_dfs = read_sub_routine(file_7z, first_date, last_date, "message", level=level, path=path)
    orderbook_dfs = read_sub_routine(file_7z, first_date, last_date, "orderbook", level=level, path=path)
    frames = []

    assert list(message_dfs.keys()) == list(orderbook_dfs.keys()), "the messages and orderbooks have different days!!"
    
    for d in message_dfs.keys():
        tmp_df = lobster_to_sec_df(
            message_dfs[d], orderbook_dfs[d], d, granularity=granularity,
            level=level, add_messages=add_messages)
        frames.append(tmp_df)

    result = pd.concat(frames, ignore_index=False)

    return result

In [7]:
df = from_folder_to_unique_df("/home/ema/dev/shocks/data/lobster/AAPL", level=10)


/tmp/ipykernel_4923/1446641199.py:41: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(curr, names=columns)
/tmp/ipykernel_4923/1446641199.py:41: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(curr, names=columns)
/tmp/ipykernel_4923/4151154403.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orderbook_df["date"] = [start_date + timedelta(seconds=i) for i in orderbook_df["seconds"]]
/tmp/ipykernel_4923/4151154403.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [2]:
import pandas as pd
pd.read_csv("/home/ema/dev/shocks/data/lobster/AAPL/2021-11-01.csv")

,date,psell1,vsell1,pbuy1,vbuy1,psell2,vsell2,pbuy2,vbuy2,psell3,...,vsell10,pbuy10,vbuy10,time,event_type,order_id,size,price,direction,unk
0,2021-11-01 09:30:00,1489500.0,50.0,1488500.0,651.0,1489700.0,1000.0,1488300.0,100.0,1490000.0,...,577.0,1487500.0,324.0,34200.001464,1.0,17802745.0,1000.0,1489700.0,-1.0,UBSS
1,2021-11-01 09:30:01,1489500.0,69.0,1488600.0,694.0,1489600.0,13.0,1488500.0,651.0,1489700.0,...,100.0,1487500.0,324.0,34201.077774,5.0,0.0,69.0,1489400.0,1.0,UBSS
2,2021-11-01 09:30:02,1489700.0,100.0,1489300.0,5.0,1489800.0,200.0,1489000.0,85.0,1489900.0,...,10.0,1487800.0,1061.0,34202.017820,1.0,18873425.0,5.0,1489300.0,1.0,UBSS
3,2021-11-01 09:30:03,1490000.0,70.0,1489600.0,100.0,1490900.0,100.0,1489400.0,133.0,1491100.0,...,679.0,1488400.0,200.0,34203.001507,5.0,0.0,70.0,1490000.0,1.0,NaN
4,2021-11-01 09:30:04,1490000.0,1424.0,1489600.0,712.0,1490900.0,100.0,1489300.0,116.0,1491100.0,...,882.0,1488200.0,41.0,34204.000023,5.0,0.0,1.0,1490000.0,1.0,UBSS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23395,2021-11-01 15:59:55,1489300.0,1930.0,1489200.0,100.0,1489400.0,1723.0,1489100.0,547.0,1489500.0,...,32349.0,1488300.0,300.0,57595.002412,4.0,429168805.0,100.0,1489300.0,-1.0,UBSS
23396,2021-11-01 15:59:56,1489500.0,250.0,1489400.0,900.0,1489600.0,24990.0,1489300.0,452.0,1489700.0,...,906.0,1488500.0,1252.0,57596.002650,4.0,429267469.0,100.0,1489500.0,-1.0,UBSS
23397,2021-11-01 15:59:57,1489200.0,87.0,1489100.0,400.0,1489300.0,237.0,1489000.0,253.0,1489400.0,...,15876.0,1488200.0,105.0,57597.002528,4.0,429366037.0,100.0,1489200.0,-1.0,NaN
23398,2021-11-01 15:59:58,1488900.0,150.0,1488700.0,1055.0,1489000.0,5700.0,1488600.0,101.0,1489100.0,...,1140.0,1487800.0,101.0,57598.003279,1.0,429477605.0,100.0,1488700.0,1.0,UBSS
